
| **Variable**                                                    | **Description**                                                     |
|-----------------------------------------------------------------|---------------------------------------------------------------------|
| R_fighter                                                       | favorite fighter in match up                                        |
| B_fighter                                                       | underdog fighter in match up                                        |
| date                                                            | when the fight took place                                           |
| location                                                        | city where the fight took place                                     |
| country                                                         | country where the fight took place                                  |
| winner                                                          | which fighter won the fight                                         |
| weight_class                                                    | weight class where the fight took place in                          |
| gender                                                          | gender of the fighters                                              |
| R/B_Stance                                                      | fight stance of the underdog/favorite fighter                       |
| finish                                                          | how did the fight end                                               |
| finish_details                                                  | specifically how did the fight end                                  |
| finish_round_time                                               | time in the round when the fight ended                              |
| title_bout                                                      | was the fight a title bout                                          |
| R/B_odds                                                        | betting odds of the favorite/underdog                               |
| R/B_ev                                                          | payout for favorite/underdog win                                    |
| no_of_rounds                                                    | number of rounds for the fight                                      |
| R/B_current_lose_streak                                         | number of consecutive losses at time of fight                       |
| R/B_current_win_streak                                          | number of consecutive wins at time of fight                         |
| R/B_draw                                                        | total number of draw up until time of fight                         |
| R/B_losses                                                      | total number of losses up until time of fight                       |
| R/B_wins                                                        | total number of wins up until time of fight                         |
| R/B_longest_win_streak                                          | longest win streak in career up until time of fight                 |
| R/B_avg_SIG_STR_landed                                          | average number of significant strikes landed per minute             |
| R/B_avg_SIG_STR_pct                                             | accuracy of significant strikes landed vs thrown                    |
| R/B_avg_SUB_ATT                                                 | average number of submission attempts in a fight                    |
| R/B_avg_TD_landed                                               | average number of takedowns landed per minute                       |
| R/B_avg_TD_pct                                                  | accuracy of takedown landed vs attempted                            |
| R/B_Height_cms                                                  | height of respective fighter in match up                            |
| R/B_Reach_cms                                                   | reach of respective fighter in match up                             |
| R/B_weight_lbs                                                  | weight of respective fighter in match up                            |
| R/B_age                                                         | age of respective fighter in match up                               |
| R/B_total_title_bouts                                           | total number of in career up until the time of fight                |
| R/B_total_round_fought                                          | total number of rounds fought in the UFC up until fight             |
| R/B_win_by_decision                                             | total number of wins respectively by decision                       |
| R/B_win_by_submission                                           | total number of wins respectively by submission                     |
| R/B_win_by_KO.TKO                                               | total number of wins respectively by knockout                       |
| R/B_win_by_TKO_Doctor                                           | total number of wins respectively by doctor stoppage                |
| win_dif                                                         | total win differential b/t respective fighter                       |
| loss_dif                                                        | total loss differential b/t respective fighter                      |
| lose_streak_dif                                                 | losing streak differential b/t respective fighter                   |
| win_streak_dif                                                  | win streak differential b/t respective fighter                      |
| longest_win_streak_dif                                          | longest win streak differential b/t respective fighter              |
| total_round_dif                                                 | differential b/t respective fighter on total rounds fought          |
| total_title_bout_dif                                            | differential b/t fighters on total title bouts fought               |
| ko_dif                                                          | differential on total knockouts + doctor stoppage wins              |
| sub_dif                                                         | differential on total submission wins                               |
| sig_str_dif                                                     | differential on significant strikes landed                          |
| avg_sub_att_dif                                                 | differential on average submission attempts                         |
| avg_td_dif                                                      | differential on takedowns landed                                    |
| empty_arena                                                     | was the fight held in an empty arena                                |
| R/B_weightclass_rank                                            | weight class rank for respective fighter                            |
| finish_round                                                    | which round did the fight end                                       |
| total_fight_time_secs                                           | total length of fight in seconds                                    |
| R/B_kd_bout                                                     | average number of knockdowns in a bout                              |
| R/B_sig_str_landed_bout                                         | average number of significant strikes landed in a bout              |
| R/B_sig_str_attempted_bout                                      | average number of significant strikes attempted in a bout           |
| R/B_sig_str_pct_bout                                            | average accuracy of significant strikes landed in a bout            |
| R/B_tot_str_landed_bout                                         | average total number of strikes landed in a bout                    |
| R/B_tot_str_attempted_bout                                      | average total number of strikes attempted in a bout                 |
| R/B_td_landed_bout                                              | average number of takedowns landed in a bout                        |
| R/B_td_attempted_bout                                           | average number of takedowns attempted in a bout                     |
| R/B_td_pct_bout                                                 | average accuracy of takedowns landed in a bout                      |
| R/B_sub_attempts_bout                                           | average number of submissions attempted in a bout                   |
| R/B_pass_bout                                                   | average number of guard-passes in a bout                            |
| R/B_rev_bout                                                    | average number of reversals (i.e., sweeps) in a bout                |

In [1]:
import pandas as pd
import psycopg2

In [2]:
class PostgresSql:
    
  def __init__(self):
    conn = psycopg2.connect(
        host="postgresdns",
        database="postgres",
        user="postgres",
        password="postgres")
    cursor = conn.cursor()
    self.cursor = cursor

  def exec(self, query, cursor=None, to_df=False):
    c = self.cursor if not cursor else cursor
    c.execute(query)
    chunk = c.fetchall()
    if to_df:
        names = list(map(lambda x : x[0], c.description))
        return pd.DataFrame(chunk, columns=names)
    
    return chunk

In [3]:
postgressql = PostgresSql()

In [4]:
master_query = """
    select * 
    from ufc_master
"""

original_df = postgressql.exec( master_query, to_df = True )
original_df.rename( inplace = True, 
                  columns = { column : column.lower() for column in original_df.columns } )

In [5]:
df = original_df.copy()

In [6]:
cols = [
    'r_odds',
    'b_odds',
    'winner'
]

In [7]:
df['winner'].nunique()

2

In [8]:
df[cols].isna().sum()

r_odds    1
b_odds    0
winner    0
dtype: int64

In [9]:
name = df[df['r_odds'].isna()]
name = name['r_fighter'].values[0]
name

'Rong Zhu'

In [10]:
df[(df['r_fighter'] == name) | (df['b_fighter'] == name)]

,r_fighter,b_fighter,r_odds,b_odds,r_ev,b_ev,date,location,country,winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
32,Rong Zhu,Brandon Jenkins,NaN,195,NaN,195,2021-09-18,"Las Vegas, Nevada, USA",USA,Red,...,Punches,3.0,4:35,875.0,NaN,500.0,NaN,1600.0,NaN,450.0
228,Kazula Vargas,Rong Zhu,175.0,-210,175.0,47,4/24/2021,"Jacksonville, Florida, USA",USA,Red,...,,NaN,,NaN,450.0,NaN,900.0,NaN,550.0,NaN


In [11]:
n = df[(df['r_fighter'] == name)]['r_odds'].sum()
m = df[(df['b_fighter'] == name)]['b_odds'].sum()

avg = n + m

df['r_odds'].fillna(avg, inplace=True)

In [12]:
df[cols].isna().sum()

r_odds    0
b_odds    0
winner    0
dtype: int64

In [13]:
def odds_favorite(r):
    r_odds, b_odds = r['r_odds'], r['b_odds']
    return 'Red' if r_odds < b_odds else 'Blue'

df = df[cols]
df['odds_winner'] = df.apply(odds_favorite, axis=1)
df

,r_odds,b_odds,winner,odds_winner
0,-150.0,130,Red,Red
1,170.0,-200,Blue,Blue
2,110.0,-130,Blue,Blue
3,-675.0,475,Red,Red
4,-135.0,115,Blue,Red
...,...,...,...,...
4891,-155.0,135,Blue,Red
4892,-210.0,175,Red,Red
4893,-260.0,220,Red,Red
4894,-420.0,335,Red,Red


In [14]:
df.drop(columns=['r_odds', 'b_odds'], inplace=True, errors='ignore')
df

,winner,odds_winner
0,Red,Red
1,Blue,Blue
2,Blue,Blue
3,Red,Red
4,Blue,Red
...,...,...
4891,Blue,Red
4892,Red,Red
4893,Red,Red
4894,Red,Red


In [15]:
df['winner'] = df['winner'].astype('category')
df['odds_winner'] = df['odds_winner'].astype('category')

df['winner_code'] = df.winner.cat.codes
df['odds_winner_code'] = df.odds_winner.cat.codes

df.head()

,winner,odds_winner,winner_code,odds_winner_code
0,Red,Red,1,1
1,Blue,Blue,0,0
2,Blue,Blue,0,0
3,Red,Red,1,1
4,Blue,Red,0,1


In [16]:
s = len(df[df['winner_code'] == df['odds_winner_code']])
l = len(df)
print(s/l)

0.6495098039215687


In [17]:
list(filter(lambda x : 'dif' in x, original_df.columns))

['lose_streak_dif',
 'win_streak_dif',
 'longest_win_streak_dif',
 'win_dif',
 'loss_dif',
 'total_round_dif',
 'total_title_bout_dif',
 'ko_dif',
 'sub_dif',
 'height_dif',
 'reach_dif',
 'age_dif',
 'sig_str_dif',
 'avg_sub_att_dif',
 'avg_td_dif']

In [18]:
cols = [
    'r_reach_cms',
    'b_reach_cms',
    'reach_dif'
]

original_df[cols]

,r_reach_cms,b_reach_cms,reach_dif
0,193.04,208.28,15.24
1,193.04,193.04,0.00
2,195.58,195.58,0.00
3,182.88,177.80,-5.08
4,177.80,172.72,-5.08
...,...,...,...
4891,177.80,180.34,2.54
4892,180.34,187.96,7.62
4893,198.12,196.00,-2.12
4894,177.80,185.42,7.62


In [21]:
xs = filter (lambda x : 'stance' in x, list(original_df.columns))

list(xs)

['b_stance', 'r_stance']

In [ ]:
cols = [
    'b_stance',
]

original_df[cols]